In [35]:
#pip install --user bertopic
#pip install --user kagglehub
#pip install --user pydantic spacy==3.3.3
#pip install --user https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.0.0/en_core_web_sm-3.0.0.tar.gz
#pip install --user bertopic
#pip install pandas elasticsearch


     ------------------------------------- 571.2/571.2 kB 18.1 MB/s eta 0:00:00
     ---------------------------------------- 64.4/64.4 kB ? eta 0:00:00


In [13]:
import requests
import json
import pandas as pd
import numpy as np
import spacy
from tqdm import tqdm
from gensim import corpora, models 
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import nltk


In [2]:
#example of an api pull from acled dataset - I don't currently have api data access so will load a csv instead
'''
# to make future prototyping updates easier re: query, make a dictionary of parameters
parameters = {
  "email": "616316@bah.com",
  "key": "mHP-cdgOAZfrlf4MJ70p",
  "country": "Niger|Nigeria|Chad|Cameroon", 
  "year": 2023,
  "fields": "event_id_cnty|event_date|event_type|country|fatalities|notes|tags"
}
# Request the data as a JSON file and pass our paramenters as an argument (params=)
response_params_dict = requests.get("https://api.acleddata.com/acled/read", params= parameters)
if response_params_dict.json()['status'] == 200:
  print("Complete request -- Returning a JSON")
else:
    print(response_params_dict.json())
    
'''#{'status': 403, 'success': False, 'count': 0, 'error': {'status': 403, 'message': 'Your account is restricted to only downloading the data via the Data Export Tool. Please contact access@acleddata.com to discuss extending your access to the API.'}}

'\n# to make future prototyping updates easier re: query, make a dictionary of parameters\nparameters = {\n  "email": "616316@bah.com",\n  "key": "mHP-cdgOAZfrlf4MJ70p",\n  "country": "Niger|Nigeria|Chad|Cameroon", \n  "year": 2023,\n  "fields": "event_id_cnty|event_date|event_type|country|fatalities|notes|tags"\n}\n# Request the data as a JSON file and pass our paramenters as an argument (params=)\nresponse_params_dict = requests.get("https://api.acleddata.com/acled/read", params= parameters)\nif response_params_dict.json()[\'status\'] == 200:\n  print("Complete request -- Returning a JSON")\nelse:\n    print(response_params_dict.json())\n    \n'

In [3]:
#grabbing the CNN news dataset from Kaggle
import kagglehub

# Download latest version
path = kagglehub.dataset_download("hadasu92/cnn-articles")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\kcsof\.cache\kagglehub\datasets\hadasu92\cnn-articles\versions\1


In [4]:
df = pd.read_csv('C:/Users/kcsof/.cache/kagglehub/datasets/hadasu92/cnn-articles/versions/1/CNN_Articles.csv')

In [5]:
#Explore the dataset
#df.head()

In [6]:
# Check for duplicate rows and remove them 
#df['url'].value_counts()
df.drop_duplicates(subset=['url'], keep="first", inplace=True)
df['url'].value_counts()

https://www.cnn.com/2021/07/14/world/tusimple-autonomous-truck-spc-intl/index.html                               1
https://www.cnn.com/2021/07/17/motorsport/ana-carrasco-supersport-broken-back-return-cmd-spt-intl/index.html     1
https://www.cnn.com/2021/08/10/australia/australia-magpie-baby-death-intl-hnk/index.html                         1
https://www.cnn.com/2021/08/10/sport/india-olympics-glory-intl-hnk/index.html                                    1
https://www.cnn.com/2021/08/09/us/lamar-jackson-baltimore-ravens-covid/index.html                                1
                                                                                                                ..
https://www.cnn.com/2021/03/12/uk/meghan-race-britain-gbr-intl/index.html                                        1
https://www.cnn.com/2021/03/11/europe/nazi-guard-trial-harry-s-stutthof-scli-intl-grm/index.html                 1
https://www.cnn.com/2021/03/11/football/nba-meyers-leonard-fine-suspension-spt-i

In [7]:
#check for null values
df.isnull().sum() #if there were I'd drop or fillna depending on the dataset

Unnamed: 0              0
author                  0
date_published          0
part_of                 0
article_section         0
url                     0
headline                0
description             0
keywords                0
alternative_headline    0
text                    0
dtype: int64

In [8]:
df.head()

,Unnamed: 0,author,date_published,part_of,article_section,url,headline,description,keywords,alternative_headline,text
0,0,"Jacopo Prisco, CNN",2021-07-15T02:46:59Z,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d..."
2,2,"Stephanie Bailey, CNN",2021-05-12T07:52:09Z,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...
3,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16T02:51:30Z,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut..."
4,4,"Paul R. La Monica, CNN Business",2022-03-15T09:57:36Z,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor..."
7,7,Reuters,2022-03-15T11:27:02Z,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...


In [9]:
#load the spaCy model
nlp = spacy.load("en_core_web_sm")

C:\Users\kcsof\AppData\Roaming\Python\Python39\site-packages\spacy\util.py:837: UserWarning: [W095] Model 'en_core_web_sm' (3.0.0) was trained with spaCy v3.0 and may not be 100% compatible with the current version (3.3.3). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
C:\Users\kcsof\AppData\Roaming\Python\Python39\site-packages\torch\__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\tensor\python_tensor.cpp:434.)
  _C._set_default_tensor_type(t)


In [10]:
#make a function to extract entities
# Function to extract entities 
def extract_entities(text):
    doc = nlp(text)
    people = []
    places = []
    companies = []
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            people.append(ent.text)
        elif ent.label_ in ['GPE', 'LOC']: 
            places.append(ent.text) 
        elif ent.label_ == 'ORG':
            companies.append(ent.text) 
    return pd.Series([people, places, companies])

In [11]:
# Apply the function to the DataFrame 
tqdm.pandas() #progress bar to check function completion progress
df[['People', 'Places', 'Companies']] = df['text'].apply(extract_entities)

print("DataFrame with extracted entities:")
print(df.head())

DataFrame with extracted entities:
   Unnamed: 0                                             author  \
0           0                                 Jacopo Prisco, CNN   
2           2                              Stephanie Bailey, CNN   
3           3  Words by Stephanie Bailey, video by Zahra Jamshed   
4           4                    Paul R. La Monica, CNN Business   
7           7                                            Reuters   

         date_published   part_of article_section  \
0  2021-07-15T02:46:59Z      news           world   
2  2021-05-12T07:52:09Z      news           world   
3  2021-06-16T02:51:30Z      news            asia   
4  2022-03-15T09:57:36Z  business       investing   
7  2022-03-15T11:27:02Z  business        business   

                                                 url  \
0  https://www.cnn.com/2021/07/14/world/tusimple-...   
2  https://www.cnn.com/2021/05/12/world/ironhand-...   
3  https://www.cnn.com/2021/06/15/asia/swarm-robo...   
4  https://ww

In [15]:
##Topic Modeling
# Download NLTK data 
nltk.download('punkt') 
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kcsof\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kcsof\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
# Preprocess the text 
stop_words = set(stopwords.words('english')) 
df['Processed_Text'] = df['text'].apply(lambda x: [word for word in word_tokenize(x.lower()) if word.isalnum() and word not in stop_words])

In [22]:
# Create a dictionary and corpus 
dictionary = corpora.Dictionary(df['Processed_Text']) 
corpus = [dictionary.doc2bow(text) for text in df['Processed_Text']]

In [23]:
# Run LDA model 
lda_model = models.LdaModel(corpus, num_topics=3, id2word=dictionary, passes=15)

# Get the top words for each topic 
topics = lda_model.show_topics(formatted=False)

# Create a mapping of topic numbers to top words 
topic_words = {i: [word for word, _ in topic] for i, topic in topics}

In [24]:
# Make a function to get the dominant topic 
def get_dominant_topic(bow): 
    topics = lda_model.get_document_topics(bow) 
    dominant_topic = max(topics, key=lambda x: x[1])[0] 
    return ', '.join(topic_words[dominant_topic])

In [25]:
# Apply the function to the DataFrame 
df['Dominant_Topic'] = df['Processed_Text'].apply(lambda x: get_dominant_topic(dictionary.doc2bow(x)))
df.head() #not ideal topic extraction, let's try bert instead

,Unnamed: 0,author,date_published,part_of,article_section,url,headline,description,keywords,alternative_headline,text,People,Places,Companies,Processed_Text,Dominant_Topic
0,0,"Jacopo Prisco, CNN",2021-07-15T02:46:59Z,news,world,https://www.cnn.com/2021/07/14/world/tusimple-...,"There's a shortage of truckers, but TuSimple t...",The e-commerce boom has exacerbated a global t...,"world, There's a shortage of truckers, but TuS...","There's a shortage of truckers, but TuSimple t...","(CNN)Right now, there's a shortage of truck d...","[Navistar, Hardt, Uber, Steve Lee, Alstom, Che...","[US, San Diego, US, China, Europe, US, Traton,...","[TuSimple, Volkswagen, Oceanbird, Maglev, Delf...","[cnn, right, shortage, truck, drivers, us, wor...","said, people, one, says, photos, caption, cnn,..."
2,2,"Stephanie Bailey, CNN",2021-05-12T07:52:09Z,news,world,https://www.cnn.com/2021/05/12/world/ironhand-...,Bioservo's robotic 'Ironhand' could protect fa...,Working in a factory can mean doing the same t...,"world, Bioservo's robotic 'Ironhand' could pro...",A robotic 'Ironhand' could protect factory wor...,(CNN)Working in a factory or warehouse can me...,"[Ocado, Ava Robotics, Stretch, Mikael Wester, ...","[Ironhand, London, Telexistence, Tokyo, Taastr...","[Bioservo, MIT, the Greater Boston Food Bank, ...","[cnn, working, factory, warehouse, mean, task,...","said, people, one, says, photos, caption, cnn,..."
3,3,"Words by Stephanie Bailey, video by Zahra Jamshed",2021-06-16T02:51:30Z,news,asia,https://www.cnn.com/2021/06/15/asia/swarm-robo...,This swarm of robots gets smarter the more it ...,"In a Hong Kong warehouse, a swarm of autonomou...","asia, This swarm of robots gets smarter the mo...",This swarm of robots gets smarter the more it ...,"(CNN)In a Hong Kong warehouse, a swarm of aut...","[Ocado, Ava Robotics, Stretch, Geek+, Lit Fung]","[Hong Kong, London, Telexistence, Tokyo, Taast...","[The Autonomous Mobile Robots, AI, MIT, the Gr...","[cnn, hong, kong, warehouse, swarm, autonomous...","said, people, one, says, photos, caption, cnn,..."
4,4,"Paul R. La Monica, CNN Business",2022-03-15T09:57:36Z,business,investing,https://www.cnn.com/2022/03/15/investing/brics...,Russia is no longer an option for investors. T...,"For many years, the world's most popular emerg...","investing, Russia is no longer an option for i...",Russia is no longer an option for investors. T...,"New York (CNN Business)For many years, the wor...","[Eric Winograd, Winograd, Mychal Campos, Winog...","[New York, Brazil, Russia, India, China, South...","[CNN, BRICS, BRICS, AllianceBernstein, MTS, th...","[new, york, cnn, business, many, years, world,...","said, people, one, says, photos, caption, cnn,..."
7,7,Reuters,2022-03-15T11:27:02Z,business,business,https://www.cnn.com/2022/03/15/business/russia...,Russian energy investment ban part of new EU s...,The European Union formally approved on Tuesda...,"business, Russian energy investment ban part o...",EU bans investment in Russian energy in new sa...,The European Union formally approved on Tuesda...,"[Roman Abramovich, Transneft, Putin, Rishi Sun...","[Russia, Ukraine, Russia, Russia, Russia, Russ...","[The European Union, EU, Chelsea, The European...","[european, union, formally, approved, tuesday,...","said, cnn, people, government, peng, country, ..."


In [27]:
from bertopic import BERTopic

# Initialize BERTopic model 
topic_model = BERTopic()

In [29]:
# Fit the model and transform the text 
topics, _ = topic_model.fit_transform(df['text']) 
# Add the topics to the DataFrame 
df['Topic'] = topics

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
# Function to get the topic words 
def get_topic_words(topic_model, topic):
    if topic == -1:
        return "No dominant topic"
    topic_words = topic_model.get_topic(topic) 
    return ', '.join([word for word, _ in topic_words])

In [32]:
# Apply the function to the DataFrame
df['Topic'] = topics
df['Topic_Words'] = df['Topic'].apply(lambda x: get_topic_words(topic_model, x))

In [33]:
print("DataFrame with BERT detected topics:") 
print(df)

DataFrame with BERT detected topics:
      Unnamed: 0                                             author  \
0              0                                 Jacopo Prisco, CNN   
2              2                              Stephanie Bailey, CNN   
3              3  Words by Stephanie Bailey, video by Zahra Jamshed   
4              4                    Paul R. La Monica, CNN Business   
7              7                                            Reuters   
...          ...                                                ...   
4725        4725                                   Story by Reuters   
4726        4726                           Homero De la Fuente, CNN   
4727        4727                                  Amy Woodyatt, CNN   
4728        4728                                   Story by Reuters   
4729        4729                 Analysis by Stephen Collinson, CNN   

            date_published   part_of article_section  \
0     2021-07-15T02:46:59Z      news           world  

In [34]:
#get the df columns so you can generate the es mappings
df.columns

Index(['Unnamed: 0', 'author', 'date_published', 'part_of', 'article_section',
       'url', 'headline', 'description', 'keywords', 'alternative_headline',
       'text', 'People', 'Places', 'Companies', 'Processed_Text',
       'Dominant_Topic', 'Topic', 'Topic_Words'],
      dtype='object')

In [79]:
##Push the data to the es index
from elasticsearch import Elasticsearch, helpers
from elasticsearch.helpers import bulk

In [80]:
#make a function to generate the es docs
def generate_docs(df): 
    for index, row in df.iterrows():
        yield { "_index": "kcorn-test-index",
               "_id": index,
               "_source": row.to_dict(),
              }


In [81]:
# Generate documents
docs = list(generate_docs(df)) 
print("Generated documents") 


Generated documents


In [69]:
# Connect to Elasticsearch 
try: 
    es = Elasticsearch('https://9d786a11020a493e94c2198ae5397ffb.us-central1.gcp.cloud.es.io:443', api_key="my_key") 
    # Push DataFrame to Elasticsearch 
    bulk(es, generate_docs(df)) 
    print("DataFrame pushed to Elasticsearch index successfully!") 
except ConnectionError as e: 
    print(f"ConnectionError: {e}")

DataFrame pushed to Elasticsearch index successfully!
